# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForQuestionAnswering, default_data_collator
from transformers import EarlyStoppingCallback
from tqdm.auto import tqdm
import collections

In [ ]:
train_dataset = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')

# My overall idea is highly generated from [this](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) awesome notebook written by 🤗

I tried to use my own understanding from this notebook and jotted things down as I move from here.

#### The few other kernels that helped me with understanding this whole code
- [kernel 1](https://www.kaggle.com/yogeshkulkarni/chai-qna-huggingface-transformers)
- [kernel 2](https://www.kaggle.com/rizdelhi/chaii-hindi-tamil-queans)


# Preparing Training and Validation Dataset

In [ ]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }


train_dataset['answers'] = train_dataset[['answer_start', 'answer_text']].apply(get_answers, axis=1)

train_dataset['answer_end'] = None
for i in range(len(train_dataset)):
    train_dataset['answer_end'][i] = (train_dataset['answer_start'][i]) + len(train_dataset['answer_text'][i])

pd.set_option('display.max_colwidth',None)


In [ ]:
len('नेताजी')

In [ ]:
train_dataset.tail(1)

In [ ]:
# this is the max sentence length that the model supports
max_length = 384

# when splitting long context this is needed to authorize overlap
doc_stride = 128

In [ ]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq # added to input from kaggle

we have to restart the kernel to have the following imports work

In [ ]:
import datasets
from datasets import Dataset

In [ ]:
df_train = train_dataset[:-64].reset_index(drop=True)
df_valid = train_dataset[-64:].reset_index(drop=True)

train = Dataset.from_pandas(df_train)
valid = Dataset.from_pandas(df_valid)
print(df_train.shape,df_valid.shape)
print(train.shape, valid.shape)

In [ ]:
valid, train

### Initializing the Tokenizer

In [ ]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("../input/hugging-face-hub-xlmrobertalargesquad2/xlm-roberta-large-squad2")

In [ ]:
tokenizer

In [ ]:
type(train.features)

# Practice with the Tokenizer

### you can skip to header **Preprocessing** if not required 😆 

we have very long contexts in this dataset. So preprocessing of these long contexts are necessary. The max length for the model that I am using is 384 so anything longer than that is not accepted. But trucation can result in loss of information and we may loose the answer to our question as well. So we will divide the long contexts shorter than the max length of our model but just in case the answer lies in the point where we split the context we will use something called overlap.

below I fetch a row with long context (longer than max length of ur model)

In [ ]:
for i , example in enumerate(train):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = train[i]

A tokenizer has 2 parts.
- input_ids
- attention_mask

I am fetching the input_ids and getting the total length of the tokenizer

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])
#(tokenizer(example["question"], example["context"])["input_ids"])

This length is definitely longer than the max length of the model so now we truncate.
But by truncation we loose a lot of information, as we can see from 1504 -> 384 (see below)

We have to remember that we only want to truncate the context not the question. Thats why the option
`only_second` has been chosen

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

just so we dont loose any information by truncation we are going to allow some overlaps as mentioned earlier. 
this can be done only using `return_overflowing_tokens`

This has been shown in th cell below:

In [ ]:
tokenizer_with_truncation_and_overlapping = tokenizer(
    example['question'],
    example['context'],
    max_length= max_length,
    truncation= 'only_second',
    return_overflowing_tokens= True,
    stride = doc_stride
)

here we see 6 dimension to `input_ids` because the overlapping has been done 6 times

In [ ]:
len(tokenizer_with_truncation_and_overlapping['input_ids'])

calculating the length of each `input_ids`

In [ ]:
[len(x) for x in tokenizer_with_truncation_and_overlapping['input_ids']]

if we decode we can see the overlap

In [ ]:
for x in tokenizer_with_truncation_and_overlapping["input_ids"]:
    print(tokenizer.decode(x))

Now we have to find out where exactly in these features the answer lies. 

**the model that we are using requires the start and end positions of these answers in the tokens.**

For this we use **mapping** which can be done using our tokenizer by returning `offset_mapping`

In [ ]:
tokenizer_with_truncation_and_overlapping = tokenizer(
    example['question'],
    example['context'],
    max_length= max_length,
    truncation= 'only_second',
    return_overflowing_tokens= True,
    return_offsets_mapping= True,
    stride = doc_stride
)

In [ ]:
len(tokenizer_with_truncation_and_overlapping['offset_mapping'])

In [ ]:
# printitng index: 0 till 99
tokenizer_with_truncation_and_overlapping['offset_mapping'][0][:100]

# now fetching the first token id and first offset mapping within this index 0
first_token_id = tokenizer_with_truncation_and_overlapping['input_ids'][0][1]
offset = tokenizer_with_truncation_and_overlapping['offset_mapping'][0][1]
print(first_token_id, offset )

offset: corresponding start and end character in the original text that gave as the token

first_token_id: the id of the generated token from the original text

In [ ]:
print(tokenizer.convert_ids_to_tokens([first_token_id]), example["question"][offset[0]:offset[1]])

But we do have to find out which part of the offset correspond to the question and which part correspond to the context. 

This is where we can use `sequesnce_ids`

it inputs **NONE** for the special tokens and 0 and 1 for the token that comes from the question and the context respectively. 

In [ ]:
sequence_id = tokenizer_with_truncation_and_overlapping.sequence_ids()
sequence_id[:15]

### Finding the first and last token of the answer in the input feature

In [ ]:
answer = example['answers']
start = answer['answer_start'][0]
end = example['answer_end']

token_start_index = 0
while sequence_id[token_start_index] != 1:
    token_start_index += 1
token_end_index = len(tokenizer_with_truncation_and_overlapping['input_ids'][0]) -1
while sequence_id[token_end_index] != 1:
    token_end_index -= 1
    
# if answer is out of the span or not
# the answer to the question is available in the 0th index of the offset, any other index iof the offset
# will not hold the answer
offsets = tokenizer_with_truncation_and_overlapping['offset_mapping'][0]
if (offsets[token_start_index][0] <= start and offsets[token_end_index][1] >= end):
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

In [ ]:
print(tokenizer.decode(tokenizer_with_truncation_and_overlapping["input_ids"][0][start_position: end_position+1]))
print(answer["text"][0])

For the model that we are using (or for it to work with any other models) padding of the tokenizer is necessary.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

# Preprocessing

Now we finally preprocess the data before feeding into our model

(putting everything that we did above together)

### Train data preprocessing

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    # sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    # offset_mapping = tokenized_examples["offset_mapping"]
    offset_mapping = tokenized_examples.pop("offset_mapping")
    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = examples['answer_end'][sample_index]

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
preprocessed_train = train.map(prepare_train_features, batched=True, remove_columns = train.column_names)

In [ ]:
preprocessed_train

In [ ]:
preprocessed_valid = valid.map(prepare_train_features, batched=True, remove_columns = train.column_names)

In [ ]:
preprocessed_valid

# Defining Model

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("../input/hugging-face-hub-xlmrobertalargesquad2/xlm-roberta-large-squad2")

In [ ]:
model.config

### Code for releasing CUDA memory

[Kernel](https://www.kaggle.com/getting-started/140636)

In [ ]:
# !pip install GPUtil
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
%env WANDB_DISABLED=True

# Preparing the Trainer

In [ ]:
data_collator = default_data_collator

In [ ]:
# Define Trainer
args = TrainingArguments(
    overwrite_output_dir=True, 
    fp16= True,
    output_dir="./output",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    warmup_steps=500,
    warmup_ratio=0.1,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=preprocessed_train,
    eval_dataset=preprocessed_valid,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
!ls -a output

In [ ]:
trainer.evaluate()

In [ ]:
# trainer.save_model("chaii-xlm-roberta-large-my-best-model-epoch-1")

In [ ]:
# !rm -rf wandb
# !rm -rf output
# !rm -rf runs
# !rm -rf chaii-xlm-roberta-large-my-best-model-epoch-1

## This is the model that we just trained using the dataset

The model below is my fine-tuned model. Later I made predictions using this fine-tuned model.

In [ ]:
checkpoint_model = AutoModelForQuestionAnswering.from_pretrained("./output/checkpoint-3480/")
checkpoint_model

In [ ]:
checkpoint_model.config

# Preparing the Test Dataset

In [ ]:
test_dataset = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')

In [ ]:
test_dataset.head(1)

In [ ]:
test_dataset.language.value_counts()

In [ ]:
test_dataset.info()

In [ ]:
test = Dataset.from_pandas(test_dataset)
test[0]

`Test` has the following columns

In [ ]:
test

## Making trainer for the `test` dataset

Using the fine-tuned model that we got by the Trainer using training/validation datasets 

In [ ]:
test_trainer = Trainer(checkpoint_model)

In [ ]:
test_trainer

# Preprocessing Test Dataset and Finalizing the Prediction

In [ ]:
def prepare_test_features_for_final_prediction(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
preprocessed_test = test.map(prepare_test_features_for_final_prediction,
                            batched = True,
                            remove_columns = test.column_names
                            )

In [ ]:
preprocessed_test

Now we want to make a smaller version of the `preprocessed_test`

In [ ]:
preprocessed_test_small = preprocessed_test.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

In [ ]:
preprocessed_test_small

In [ ]:
predictions = test_trainer.predict(preprocessed_test_small)

In [ ]:
predictions

the first array in the `predictions` is the `start logit` and the second array in `predictions` is the `end logit`

In [ ]:
predictions.predictions

## Decoding the text from this `predictions` variable is quite challenging for me to understand but I am trying

The huggingface 🤗 kernel has it all in details.

In [ ]:
def postprocess_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
prediction = postprocess_predictions(test, preprocessed_test, predictions.predictions)

In [ ]:
prediction

# Submission

In [ ]:
submission = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")

In [ ]:
submission.head()

In [ ]:
submission_ = pd.DataFrame({"id":submission["id"],"PredictionString":prediction.values()})

In [ ]:
submission_

In [ ]:
submission_.shape

In [ ]:
submission_.to_csv('submission.csv', index = False)

In [ ]:
# submission_.to_csv('./submission.csv', index = False) 